In [1]:
import sys
import os
import json

import pandas as pd
import numpy as np

from collections import defaultdict

import psycopg2
import psycopg2.extras as extras
from psycopg2 import extensions

import sqlalchemy
from sqlalchemy import create_engine

import warnings
warnings.filterwarnings('ignore')

In [2]:
root_dir = 'C:/Users/delst/OneDrive/Desktop/Code/Workspace/JSON_to_PGSQL'
sys.path.append(root_dir)

from A_Main.Configurations.setup_env import setup_environment
config = setup_environment(root_dir)

In [3]:
raw_data_path = config.raw_data

---

In [4]:
with open(config.postgres_password, 'r') as file:
    db_pw = file.read()

---

In [5]:
conn = psycopg2.connect(
    dbname="json_to_pgsql",
    user="postgres",
    password=db_pw,
    host="localhost"
)

In [6]:
cur = conn.cursor()

In [7]:
cur.execute(
    'SELECT * FROM reddit_posts'
    )
rows = cur.fetchall()

col_names = [desc[0] for desc in cur.description]

row_store = []
for row in rows:
    row_store.append(row)

df = pd.DataFrame(row_store, columns=col_names)
df.head()

,pinned,thumbnail_width,can_gild,distinguished,locked,is_created_from_ads_ui,is_original_content,top_awarded_type,mod_reports,hide_score,...,mod_note,author_is_blocked,selftext,media_only,report_reasons,mod_reason_title,author_flair_richtext,mod_reason_by,allow_live_comments,comment_ratio
0,False,140,True,None,False,False,False,None,[],False,...,None,False,,False,None,None,[],None,True,10.0
1,False,140,True,None,False,False,False,None,[],False,...,None,False,,False,None,None,[],None,False,17.0
2,False,140,True,None,False,False,False,None,[],False,...,None,False,,False,None,None,[],None,True,15.0
3,False,140,True,None,False,False,False,None,[],False,...,None,False,,False,None,None,[],None,True,17.0
4,False,140,True,None,False,False,False,None,[],False,...,None,False,,False,None,None,[],None,False,17.0


In [8]:
cur.execute(
    'SELECT secure_media FROM reddit_posts'
    )
rows = cur.fetchall()

row_store = []
for row in rows:
    row_store.append(row)

df_sec_med = pd.DataFrame(row_store, columns=['secure_media'])

In [9]:
df_sec_med

,secure_media
0,None
1,None
2,"{'reddit_video': {'bitrate_kbps': 1200, 'fallb..."
3,None
4,"{'reddit_video': {'bitrate_kbps': 2400, 'fallb..."
5,None
6,None
7,"{'reddit_video': {'bitrate_kbps': 2400, 'fallb..."
8,None
9,"{'reddit_video': {'bitrate_kbps': 2400, 'fallb..."


In [10]:
df_sec_med.secure_media.to_list()

['None',
 'None',
 "{'reddit_video': {'bitrate_kbps': 1200, 'fallback_url': 'https://v.redd.it/evmp79fbr56b1/DASH_480.mp4?source=fallback', 'has_audio': True, 'height': 480, 'width': 854, 'scrubber_media_url': 'https://v.redd.it/evmp79fbr56b1/DASH_96.mp4', 'dash_url': 'https://v.redd.it/evmp79fbr56b1/DASHPlaylist.mpd?a=1689435086%2CZGI5YzZlMTBkMWViZmZjM2E4NjZhODJlMTM3ZjVjMWNhYWI3NjM5YTcyZmE5ZjA2Mjk5OGUyZjQ5OWY3MDdiMQ%3D%3D&amp;v=1&amp;f=sd', 'duration': 68, 'hls_url': 'https://v.redd.it/evmp79fbr56b1/HLSPlaylist.m3u8?a=1689435086%2CMzA1YjFkMDUwYTIwZDkzZTZjMWVjMGZhYTJmNmI4N2Y0ZWJkYjQ1MjU2YzgwZDJiNmI4MmU0OWRhYTU4MGIyMQ%3D%3D&amp;v=1&amp;f=sd', 'is_gif': False, 'transcoding_status': 'completed'}}",
 'None',
 "{'reddit_video': {'bitrate_kbps': 2400, 'fallback_url': 'https://v.redd.it/9az2zw4ou56b1/DASH_720.mp4?source=fallback', 'height': 720, 'width': 406, 'scrubber_media_url': 'https://v.redd.it/9az2zw4ou56b1/DASH_96.mp4', 'dash_url': 'https://v.redd.it/9az2zw4ou56b1/DASHPlaylist.mpd?a=16

In [11]:
cur.execute(
    'SELECT secure_media FROM reddit_posts'
    )
rows = cur.fetchall()

row_store = []
for row in rows:
    row_store.append(row)

row_store

[('None',),
 ('None',),
 ("{'reddit_video': {'bitrate_kbps': 1200, 'fallback_url': 'https://v.redd.it/evmp79fbr56b1/DASH_480.mp4?source=fallback', 'has_audio': True, 'height': 480, 'width': 854, 'scrubber_media_url': 'https://v.redd.it/evmp79fbr56b1/DASH_96.mp4', 'dash_url': 'https://v.redd.it/evmp79fbr56b1/DASHPlaylist.mpd?a=1689435086%2CZGI5YzZlMTBkMWViZmZjM2E4NjZhODJlMTM3ZjVjMWNhYWI3NjM5YTcyZmE5ZjA2Mjk5OGUyZjQ5OWY3MDdiMQ%3D%3D&amp;v=1&amp;f=sd', 'duration': 68, 'hls_url': 'https://v.redd.it/evmp79fbr56b1/HLSPlaylist.m3u8?a=1689435086%2CMzA1YjFkMDUwYTIwZDkzZTZjMWVjMGZhYTJmNmI4N2Y0ZWJkYjQ1MjU2YzgwZDJiNmI4MmU0OWRhYTU4MGIyMQ%3D%3D&amp;v=1&amp;f=sd', 'is_gif': False, 'transcoding_status': 'completed'}}",),
 ('None',),
 ("{'reddit_video': {'bitrate_kbps': 2400, 'fallback_url': 'https://v.redd.it/9az2zw4ou56b1/DASH_720.mp4?source=fallback', 'height': 720, 'width': 406, 'scrubber_media_url': 'https://v.redd.it/9az2zw4ou56b1/DASH_96.mp4', 'dash_url': 'https://v.redd.it/9az2zw4ou56b1/DASHPlay